### Inicializando o Dataset

In [2]:
import pandas as pd
import numpy as np

# Load the DataFrame from a Parquet file
df = pd.read_parquet('examples/tce.parquet')

print(df['Elemento'].info())

qtd_maxima = len(str(df['Elemento'][0]))

print(f'\nQuantidade máxima de caracteres em df[Elemento]: {qtd_maxima}') # já foi verificado e todos possuem 17 caracteres

<class 'pandas.core.series.Series'>
RangeIndex: 1484918 entries, 0 to 1484917
Series name: Elemento
Non-Null Count    Dtype
--------------    -----
1484918 non-null  int64
dtypes: int64(1)
memory usage: 11.3 MB
None

Quantidade máxima de caracteres em df[Elemento]: 17


### Query-teste

In [53]:
df['Elemento_to_str'] = df['Elemento'].astype(str)

In [18]:
query = '69077'
string = df[df['Elemento_to_str'].str.contains(query, na=False)]['Elemento_to_str']
print(string)

348581     20180054646907700
348597     20180054646907700
348599     20180054646907700
348600     20180054646907700
348602     20180054646907700
                 ...        
1401289    20190409646907700
1401297    20200409646907700
1401302    20200409646907700
1401304    20210409646907700
1401310    20180409646907700
Name: Elemento_to_str, Length: 162, dtype: object


In [19]:
print(df['Elemento_to_str'][348581][9])
print(df['Elemento_to_str'][348581][10])
print(df['Elemento_to_str'][348581][11:13])
print(df['Elemento_to_str'][348581][13:15])

4
6
90
77


Vemos que os caracteres do código se dividem em:

- `df['Elemento_to_str'][9]` -> categoria econômica
- `df['Elemento_to_str'][10]` -> grupo de natureza da despesa
- `df['Elemento_to_str'][11:13]` -> modalidade de aplicação
- `df['Elemento_to_str'][13:15]` -> elemento da despesa

### Verificação de categorias

In [2]:
c_categorias = np.array(['3', '4'])
g_categorias = np.array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])                                                                                                     
mm_categorias = np.array(['96', '95', '92', '94', '93', '91', '90', '54', '53', '55', '51', '67', '72', '32', '42', '22', '00', '99', '71', '74', '73', '30', '31', '70', '76', '75', '60', '50', '40', '41', '80', '36', '35', '46', '45', '20'])
ee_categorias = np.array(['53', '54', '01', '62', '61', '63', '64', '38', '18', '20', '46', '19', '49', '42', '06', '90', '02', '50', '60', '86', '98', '66', '65', '04', '07', '41', '75', '73', '74', '67', '92', '81', '29', '14', '15', '27', '25', '52', '95', '93', '94', '21', '23', '37', '30', '32', '00', '51', '26', '13', '47', '34', '16', '17', '48', '08', '57', '58', '05', '24', '22', '36', '39', '84', '33', '03', '55', '56', '59', '31', '77', '76', '71', '72', '70', '28', '99', '96', '09', '10', '91', '35', '40', '45', '43', '85', '11', '12', '83', '97'])

In [21]:
# Verificação se existem números fora do padrão
c = df['Elemento_to_str'].str[9]
mask_c = ~c.isin(c_categorias)
print(f'categorias presentes que não eram para existir: {np.unique(c[mask_c])}')

# Categoria Econômica também pode haver o '0' como o 'Não informado'

categorias presentes que não eram para existir: ['0']


In [22]:
# Verificação se existem números fora do padrão
g = df['Elemento_to_str'].str[10]
mask_g = ~g.isin(g_categorias)
print(np.unique(g[mask_g]))

[]


In [23]:
# Verificação se existem números fora do padrão
mm = df['Elemento_to_str'].str[11:13]
mask_mm = ~mm.isin(mm_categorias)
print(f'categorias presentes que não existem: {np.unique(mm[mask_mm])}')

categorias presentes que não existem: ['33' '44']


In [24]:
# Verificação se existem números fora do padrão
ee = df['Elemento_to_str'].str[13:15]
mask_ee = ~ee.isin(ee_categorias)
print(f'Categorias presentes que não existem: {np.unique(ee[mask_ee])}')

Categorias presentes que não existem: ['44']


### Verificação de categorias 'Não Informado'

Essa categoria comumente estará como '0' no seu respectivo lugar dentro do código: 'cgmmee'. Aqui, verificaremos se, mesmo após o '0' estar presente em categorias superiores, as categorias inferiores estarão preenchidas.

In [25]:
c_zeros = df[df['Elemento_to_str'].str[9] == '0']
print(f'Quantidade de empenhos com Categoria Econômica igual a 0 = {len(c_zeros)}\n')


c_zeros_irreg_g = c_zeros[c_zeros['Elemento_to_str'].str[10] != '0']
c_zeros_irreg_mm = c_zeros[c_zeros['Elemento_to_str'].str[11:13] != '00']
c_zeros_irreg_ee = c_zeros[c_zeros['Elemento_to_str'].str[13:15] != '00']

print(f'Destes, {len(c_zeros_irreg_g)} (empenhos) possuem dígito diferente de 0 para grupo_natureza_despesa ')
print(f'Destes, {len(c_zeros_irreg_mm)} (empenhos) possuem dígito diferente de 0 para modalidade_de_aplicaçao')
print(f'Destes, {len(c_zeros_irreg_ee)} (empenhos) possuem dígito diferente de 0 para elemento_da_despesa\n')

print(f'Exemplo:\n {c_zeros_irreg_mm['Elemento_to_str'].str[9:15].iloc[:5]}')



Quantidade de empenhos com Categoria Econômica igual a 0 = 97

Destes, 0 (empenhos) possuem dígito diferente de 0 para grupo_natureza_despesa 
Destes, 97 (empenhos) possuem dígito diferente de 0 para modalidade_de_aplicaçao
Destes, 97 (empenhos) possuem dígito diferente de 0 para elemento_da_despesa

Exemplo:
 782596     004490
1016791    003390
1016798    003390
1016799    003190
1016800    003190
Name: Elemento_to_str, dtype: object


In [26]:
g_zeros = df[df['Elemento_to_str'].str[10] == '0']
print(f'Quantidade de empenhos com Grupo de Natureza da Despesa igual a 0 = {len(g_zeros)}\n')

g_zeros_irreg_c = g_zeros[g_zeros['Elemento_to_str'].str[9] != '0']
g_zeros_irreg_mm = g_zeros[g_zeros['Elemento_to_str'].str[11:13] != '00']
g_zeros_irreg_ee = g_zeros[g_zeros['Elemento_to_str'].str[13:15] != '00']

print(f'Destes, {len(g_zeros_irreg_c)} possuem dígito diferente de 0 para Categoria_Economica')
print(f'Destes, {len(g_zeros_irreg_mm)} possuem dígito diferente de 0 para modalidade_de_aplicaçao')
print(f'Destes, {len(g_zeros_irreg_ee)} possuem dígito diferente de 0 para elemento_da_despesa\n')

print(f'Exemplo:\n {g_zeros_irreg_mm['Elemento_to_str'].str[9:15].iloc[:5]}')



Quantidade de empenhos com Grupo de Natureza da Despesa igual a 0 = 97

Destes, 0 possuem dígito diferente de 0 para Categoria_Economica
Destes, 97 possuem dígito diferente de 0 para modalidade_de_aplicaçao
Destes, 97 possuem dígito diferente de 0 para elemento_da_despesa

Exemplo:
 782596     004490
1016791    003390
1016798    003390
1016799    003190
1016800    003190
Name: Elemento_to_str, dtype: object


In [27]:
mm_zeros = df[df['Elemento_to_str'].str[11:13] == '00']
print(f'Quantidade de empenhos com Modalidade de Aplicação igual a 00 = {len(mm_zeros)}\n')

mm_zeros_irreg_c = mm_zeros[mm_zeros['Elemento_to_str'].str[9] != '0']
mm_zeros_irreg_g = mm_zeros[mm_zeros['Elemento_to_str'].str[10] != '0']
mm_zeros_irreg_ee = mm_zeros[mm_zeros['Elemento_to_str'].str[13:15] != '00']

print(f'Destes, {len(mm_zeros_irreg_c)} possuem dígito diferente de 0 para Categoria_Economica')
print(f'Destes, {len(mm_zeros_irreg_g)} possuem dígito diferente de 0 para grupo_natureza_despesa')
print(f'Destes, {len(mm_zeros_irreg_ee)} possuem dígito diferente de 0 para elemento_da_despesa\n')

# print(f'Exemplo:\n {mm_zeros_irreg_ee['Elemento_to_str'].str[9:15].iloc[:5]}')



Quantidade de empenhos com Modalidade de Aplicação igual a 00 = 0

Destes, 0 possuem dígito diferente de 0 para Categoria_Economica
Destes, 0 possuem dígito diferente de 0 para grupo_natureza_despesa
Destes, 0 possuem dígito diferente de 0 para elemento_da_despesa



In [28]:
ee_zeros = df[df['Elemento_to_str'].str[13:15] == '00']
print(f'Quantidade de empenhos com Elemento da Despesa igual a 00 = {len(ee_zeros)}\n')

ee_zeros_irreg_c = ee_zeros[ee_zeros['Elemento_to_str'].str[9] != '0']
ee_zeros_irreg_g = ee_zeros[ee_zeros['Elemento_to_str'].str[10] != '0']
ee_zeros_irreg_mm = ee_zeros[ee_zeros['Elemento_to_str'].str[11:13] != '00']

print(f'Destes, {len(ee_zeros_irreg_c)} possuem dígito diferente de 0 para Categoria_Economica')
print(f'Destes, {len(ee_zeros_irreg_g)} possuem dígito diferente de 0 para grupo_natureza_despesa')
print(f'Destes, {len(ee_zeros_irreg_mm)} possuem dígito diferente de 0 para modalidade_de_aplicaçao\n')

print(f'Exemplo:\n {ee_zeros['Elemento_to_str'].str[9:15]}')



Quantidade de empenhos com Elemento da Despesa igual a 00 = 29

Destes, 29 possuem dígito diferente de 0 para Categoria_Economica
Destes, 29 possuem dígito diferente de 0 para grupo_natureza_despesa
Destes, 29 possuem dígito diferente de 0 para modalidade_de_aplicaçao

Exemplo:
 767437     319100
767438     319100
767439     319100
767440     319100
767441     319100
767442     319100
767443     319100
767444     319100
767445     319100
767446     319100
767447     319100
767448     319100
767449     319100
767450     319100
767451     319100
767452     319100
1078842    317100
1078843    447100
1078850    317100
1078851    447100
1078852    337100
1160209    337100
1262517    335000
1262518    335000
1262519    335000
1262520    335000
1262521    335000
1396332    337100
1396333    337100
Name: Elemento_to_str, dtype: object


### Análisando sobre a coluna 'ElemDespesaTCE'

In [4]:
elemdespesa_zeros = df[df['Elemento_to_str'].str[13:15] == '00']

# Todos esses, estão como '00' na coluna 'Elemento', no entanto, na coluna 'ElemDespesaTCE', eles possuem nome:
print(np.unique(elemdespesa_zeros['ElemDespesaTCE']))

['CONTRIBUICOES PREVIDENCIARIAS   RPPS   PESSOAL ATIVO   PLANO PREVIDENCIARIO'
 'TRANSFERENCIAS A CONSORCIOS PUBLICOS MEDIANTE CONTRATO DE RATEIO'
 'TRANSFERENCIAS A INSTITUICOES PRIVADAS SEM FINS LUCRATIVOS']


In [3]:
import pandas as pd

data = [
    ("01", "Aposentadorias, Reserva Remunerada e Reformas"),
    ("03", "Pensões"),
    ("04", "Contratação por Tempo Determinado"),
    ("06", "Benefício Mensal ao Deficiente e ao Idoso"),
    ("07", "Contribuição a Entidades Fechadas de Previdência"),
    ("08", "Outros Benefícios Assistenciais do Servidor e do Militar"),
    ("10", "Seguro Desemprego e Abono Salarial"),
    ("11", "Vencimentos e Vantagens Fixas – Pessoal Civil"),
    ("12", "Vencimentos e Vantagens Fixas – Pessoal Militar"),
    ("13", "Obrigações Patronais"),
    ("14", "Diárias – Civil"),
    ("15", "Diárias – Militar"),
    ("16", "Outras Despesas Variáveis – Pessoal Civil"),
    ("17", "Outras Despesas Variáveis – Pessoal Militar"),
    ("18", "Auxílio Financeiro a Estudantes"),
    ("19", "Auxílio-Fardamento"),
    ("20", "Auxílio Financeiro a Pesquisadores"),
    ("21", "Juros sobre a Dívida por Contrato"),
    ("22", "Outros Encargos sobre a Dívida por Contrato"),
    ("23", "Juros, Deságios e Descontos da Dívida Mobiliária"),
    ("24", "Outros Encargos sobre a Dívida Mobiliária"),
    ("25", "Encargos sobre Operações de Crédito por Antecipação da Receita"),
    ("26", "Obrigações decorrentes de Política Monetária"),
    ("27", "Encargos pela Honra de Avais, Garantias, Seguros e Similares"),
    ("28", "Remuneração de Cotas de Fundos Autárquicos"),
    ("29", "Distribuição de Resultado de Empresas Estatais Dependentes"),
    ("30", "Material de Consumo"),
    ("31", "Premiações Culturais, Artísticas, Científicas, Desportivas e Outras"),
    ("32", "Material, Bem ou Serviço para Distribuição Gratuita"),
    ("33", "Passagens e Despesas com Locomoção"),
    ("34", "Outras Despesas de Pessoal decorrentes de Contratos de Terceirização"),
    ("35", "Serviços de Consultoria"),
    ("36", "Outros Serviços de Terceiros – Pessoa Física"),
    ("37", "Locação de Mão-de-Obra"),
    ("38", "Arrendamento Mercantil"),
    ("39", "Outros Serviços de Terceiros – Pessoa Jurídica"),
    ("40", "Serviços de Tecnologia da Informação e Comunicação - Pessoa Jurídica"),
    ("41", "Contribuições"),
    ("42", "Auxílios"),
    ("43", "Subvenções Sociais"),
    ("45", "Subvenções Econômicas"),
    ("46", "Auxílio-Alimentação"),
    ("47", "Obrigações Tributárias e Contributivas"),
    ("48", "Outros Auxílios Financeiros a Pessoas Físicas"),
    ("49", "Auxílio-Transporte"),
    ("51", "Obras e Instalações"),
    ("52", "Equipamentos e Material Permanente"),
    ("53", "Aposentadorias do RGPS – Área Rural"),
    ("54", "Aposentadorias do RGPS – Área Urbana"),
    ("55", "Pensões do RGPS – Área Rural"),
    ("56", "Pensões do RGPS – Área Urbana"),
    ("57", "Outros Benefícios do RGPS – Área Rural"),
    ("58", "Outros Benefícios do RGPS – Área Urbana"),
    ("59", "Pensões Especiais"),
    ("61", "Aquisição de Imóveis"),
    ("62", "Aquisição de Produtos para Revenda"),
    ("63", "Aquisição de Títulos de Crédito"),
    ("64", "Aquisição de Títulos Representativos de Capital já Integralizado"),
    ("65", "Constituição ou Aumento de Capital de Empresas"),
    ("66", "Concessão de Empréstimos e Financiamentos"),
    ("67", "Depósitos Compulsórios"),
    ("70", "Rateio pela participação em Consórcio Público"),
    ("71", "Principal da Dívida Contratual Resgatado"),
    ("72", "Principal da Dívida Mobiliária Resgatado"),
    ("73", "Correção Monetária ou Cambial da Dívida Contratual Resgatada"),
    ("74", "Correção Monetária ou Cambial da Dívida Mobiliária Resgatada"),
    ("75", "Correção Monetária da Dívida de Operações de Crédito por Antecipação de Receita"),
    ("76", "Principal Corrigido da Dívida Mobiliária Refinanciado"),
    ("77", "Principal Corrigido da Dívida Contratual Refinanciado"),
    ("81", "Distribuição Constitucional ou Legal de Receitas"),
    ("82", "Aporte de Recursos pelo Parceiro Público em Favor do Parceiro Privado Decorrente de Contrato de Parceria Público-Privada - PPP"),
    ("83", "Despesas Decorrentes de Contrato de Parceria Público-Privada - PPP, exceto Subvenções Econômicas, Aporte e Fundo Garantidor"),
    ("84", "Despesas Decorrentes da Participação em Fundos, Organismos, ou Entidades Assemelhadas, Nacionais e Internacionais"),
    ("85", "Contrato de Gestão"),
    ("86", "Compensações a Regimes de Previdência"),
    ("91", "Sentenças Judiciais"),
    ("92", "Despesas de Exercícios Anteriores"),
    ("93", "Indenizações e Restituições"),
    ("94", "Indenizações e Restituições Trabalhistas"),
    ("95", "Indenização pela Execução de Trabalhos de Campo"),
    ("96", "Ressarcimento de Despesas de Pessoal Requisitado"),
    ("97", "Aporte para Cobertura do Déficit Atuarial do RPPS"),
    ("98", "Despesas do Orçamento de Investimento"),
    ("99", "A Classificar")
]

data = pd.DataFrame(data, columns=["Código", "Descrição"])
print(data)


   Código                                          Descrição
0      01      Aposentadorias, Reserva Remunerada e Reformas
1      03                                            Pensões
2      04                  Contratação por Tempo Determinado
3      06          Benefício Mensal ao Deficiente e ao Idoso
4      07   Contribuição a Entidades Fechadas de Previdência
..    ...                                                ...
79     95    Indenização pela Execução de Trabalhos de Campo
80     96   Ressarcimento de Despesas de Pessoal Requisitado
81     97  Aporte para Cobertura do Déficit Atuarial do RPPS
82     98              Despesas do Orçamento de Investimento
83     99                                      A Classificar

[84 rows x 2 columns]


In [4]:
df['Código'] = df['Elemento_to_str'].str[13:15]

df_merged = pd.merge(df, data, on="Código", how="left") # Mantendo todas as linhas de 'df', enquanto 'data', caso tiver linhas que não dêem match, não aparecerão


In [7]:
df_merged[['Descrição', 'ElemDespesaTCE']].iloc[:5]

,Descrição,ElemDespesaTCE
0,Outros Serviços de Terceiros – Pessoa Jurídica,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA
1,Serviços de Tecnologia da Informação e Comunic...,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA
2,Outros Serviços de Terceiros – Pessoa Jurídica,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA
3,Outros Serviços de Terceiros – Pessoa Jurídica,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA
4,Outros Serviços de Terceiros – Pessoa Jurídica,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA


In [5]:
import unicodedata
import re

def normalize_text(text):
    if pd.isna(text):
        return ""
    # Remove accents
    text = unicodedata.normalize('NFKD', text)
    text = ''.join([c for c in text if not unicodedata.combining(c)])
    # Lowercase
    text = text.lower()
    # Replace dashes with space
    text = re.sub(r"[-–]", " ", text)
    # Replace multiple spaces with one
    text = re.sub(r"\s+", " ", text)
    # Trim
    text = text.strip()
    return text

In [6]:
df_merged["Descricao_normalizada"] = df_merged["Descrição"].apply(normalize_text)
df_merged["ElemDespesaTCE_normalizada"] = df_merged["ElemDespesaTCE"].apply(normalize_text)

In [7]:
df_merged_reduced = df_merged[['Descricao_normalizada', 'ElemDespesaTCE_normalizada']]

In [8]:
elementos_despesa_conflitantes = df_merged_reduced[
    df_merged_reduced['Descricao_normalizada'] != df_merged_reduced['ElemDespesaTCE_normalizada']
]

In [18]:
print(f'Quantos Empenhos possuem elementos_despesa conflitantes: {len(elementos_despesa_conflitantes)}')
print(f'Porcentagem em relacao ao dataset completo: {100*(len(elementos_despesa_conflitantes)/len(df)):.4f}%')

Quantos Empenhos possuem elementos_despesa conflitantes: 247511
Porcentagem em relacao ao dataset completo: 16.6683%


In [ ]:
print(elementos_despesa_conflitantes.iloc[10:20])

# Me parece que os elementos da despesa conflitantes, se tratam, (à princípio) de pequenas diferenças no texto.
# Parece que eles estão fortemente relacionados, não sendo claro se realmente há uma diferença entre eles.

                       Descricao_normalizada  \
28         despesas de exercicios anteriores   
29         despesas de exercicios anteriores   
30  indenizacoes e restituicoes trabalhistas   
32         despesas de exercicios anteriores   
36  indenizacoes e restituicoes trabalhistas   
37         despesas de exercicios anteriores   
46         despesas de exercicios anteriores   
47         despesas de exercicios anteriores   
48  indenizacoes e restituicoes trabalhistas   
49  indenizacoes e restituicoes trabalhistas   

                           ElemDespesaTCE_normalizada  
28  desp de exercicios anteriores relativas a pess...  
29  desp de exercicios anteriores relativas a pess...  
30    indenizacoes e restituicoes trabalhistas ativos  
32  desp de exercicios anteriores relativas a pess...  
36    indenizacoes e restituicoes trabalhistas ativos  
37  desp de exercicios anteriores relativas a pess...  
46  desp de exercicios anteriores relativas a pess...  
47  desp de exercicios 

### Base do portal da transparência

In [30]:
import pandas as pd

path = '20190102_Despesas_Empenho.csv'

base_2019 = pd.read_csv(path, delimiter=';', encoding='latin1')

In [54]:
df_2019 = df[df['DtEmpenho'] == '2019-01-02']

In [15]:
print(len(df_2019))
print(len(base_2019))

16327
1204


In [33]:
print(len(np.unique(df_2019['IdEmpenho'])))
print(len(np.unique(base_2019['Id Empenho'])))

16326
1204


In [49]:
base_2019.columns

Index(['Id Empenho', 'Código Empenho', 'Código Empenho Resumido',
       'Data Emissão', 'Código Tipo Documento', 'Tipo Documento',
       'Tipo Empenho', 'Espécie Empenho', 'Código Órgão Superior',
       'Órgão Superior', 'Código Órgão', 'Órgão', 'Código Unidade Gestora',
       'Unidade Gestora', 'Código Gestão', 'Gestão', 'Código Favorecido',
       'Favorecido', 'Observação', 'Código Esfera Orçamentária',
       'Esfera Orçamentária', 'Código Tipo Crédito', 'Tipo Crédito',
       'Código Grupo Fonte Recurso', 'Grupo Fonte Recurso',
       'Código Fonte Recurso', 'Fonte Recurso', 'Código Unidade Orçamentária',
       'Unidade Orçamentária', 'Código Função', 'Função', 'Código SubFunção',
       'SubFunção', 'Código Programa', 'Programa', 'Código Ação', 'Ação',
       'Linguagem Cidadã', 'Código Subtítulo (Localizador)',
       'Subtítulo (Localizador)', 'Código Plano Orçamentário',
       'Plano Orçamentário', 'Código Programa Governo',
       'Nome Programa Governo', 'Autor Emenda'

In [50]:
df_2019.columns

Index(['index', 'Analise', 'IdEmpenho', 'Ano', 'Vlr_AnulacaoEmpenho',
       'CdFonteTCE', 'CdFonteUG', 'CNPJRaiz', 'CPFCNPJCredorQtNrs',
       'CPFCNPJCredor', 'Credor', 'DtEmpenho', 'DEFonteTCE', 'DEFonteUG',
       'DEPrograma', 'DEProjAtiv', 'DtAnomes', 'Elemento', 'ElemDespesaTCE',
       'ElemDespesaUG', 'Ente', 'Esfera', 'Funcao', 'Historico', 'IdContrato',
       'IdFonte', 'IdFuncao', 'Id_Orgao', 'IdPrograma', 'IdSubFuncao',
       'IdUnid', 'IdOrgao', 'NrFonte', 'NrFonteUG', 'NrLicitacao',
       'NrProjAtiv', 'NrEmpenho', 'ProgTrab', 'ProgTrabRed', 'ProjAtiv',
       'SubFuncao', 'Tp_Empenho', 'Unidade', 'Vlr_Empenho',
       'Vlr_Anul_Liquidacao', 'Vlr_Liquidacao', 'Vlr_Pagto', 'Vlr_Retencao',
       'Vlr_SubEmpenho', 'Vlr_Empenhado', 'Vlr_Liquidado', 'Vlr_Pago',
       'CGElem', 'CGProgTrab', 'CGigual', 'Cod_Elem', 'Cod_PT', 'CG',
       'CGtitulo', 'CGDesc', 'CGtitTCE', 'CGfreq', 'CGlevel', 'CGpai',
       'CGroot', 'CGchild'],
      dtype='object')

In [56]:
codigos_df_2019 = df_2019['Elemento_to_str'].str[9:15]
codigos_df_2019.head()

3      339039
4      339039
66     339039
67     339039
143    339014
Name: Elemento_to_str, dtype: object

In [57]:
codigos_df_2019.unique()

array(['339039', '339014', '469071', '329021', '339036', '339033',
       '339040', '339091', '339030', '339035', '449051', '339046',
       '335039', '319092', '329022', '319011', '319096', '319013',
       '339092', '339047', '332093', '319009', '319004', '319192',
       '319113', '339048', '319016', '449052', '319001', '319003',
       '339093', '319091', '319005', '449039', '337170', '317170',
       '339034', '339018', '319094', '339032', '329121', '335041',
       '339139', '339197', '469171', '339049', '319014', '339003',
       '449165', '339008', '335043', '449092', '469077', '469091',
       '339037', '319034', '339041', '332047', '337270', '447170',
       '339031', '469093', '329122', '336045', '334043', '339005',
       '339013', '449061', '336783', '333017', '449030'], dtype=object)

In [ ]:
query = str(339008)

base_2019_filtrada = base_2019[
    (base_2019['Código Categoria de Despesa'] == int(query[0])) &
    (base_2019['Código Grupo de Despesa'] == int(query[1])) &
    (base_2019['Código Modalidade de Aplicação'] == int(query[2:4])) &
    (base_2019['Código Elemento de Despesa'] == int(query[4:6]))
]

# Existem códigos de df que não existem na base_2019 do portal transparência !

len(base_2019_filtrada)


8

In [90]:
df_2019_filtrado = df_2019[df_2019['Elemento_to_str'].str[9:15] == query]

len(df_2019_filtrado)

9

In [92]:
df_2019_filtrado['Vlr_Empenho']

285779       2241.9
302935     122500.0
533371      10000.0
535471      38000.0
847261       1000.0
1008575     56000.0
1008579     14000.0
1008581     68000.0
1126894      6000.0
Name: Vlr_Empenho, dtype: float64

In [93]:
base_2019_filtrada['Valor Original do Empenho']

259     4049,91
382       69892
531       68448
532      400624
622      240000
826      150000
1075        100
1162      41988
Name: Valor Original do Empenho, dtype: object

In [110]:
base_2019['Codigo_Empenho_str'] = base_2019['Código Empenho'].astype(str)

In [119]:
query = base_2019[base_2019['Codigo_Empenho_str'].str[17:24].str.contains('800007', na=False)]
query

,Id Empenho,Código Empenho,Código Empenho Resumido,Data Emissão,Código Tipo Documento,Tipo Documento,Tipo Empenho,Espécie Empenho,Código Órgão Superior,Órgão Superior,...,Modalidade de Licitação,Inciso,Amparo,Referência de Dispensa ou Inexigibilidade,Código Convênio,Contrato de Repasse / Termo de Parceria / Outros,Valor Original do Empenho,Valor do Empenho Convertido pra R$,Valor Utilizado na Conversão,Codigo_Empenho_str
2,62402352,244001242902019NE800007,2019NE800007,02/01/2019,NE,Nota de Empenho,Estimativo,ORIGINAL,35000,Ministério das Relações Exteriores,...,Dispensa de Licitação,2,LEI8666,ART24/02 LEI8666/93,-1,SEM INFORMACAO,15400,15400,1,244001242902019NE800007
300,70202175,170145000012019NE800007,2019NE800007,02/01/2019,NE,Nota de Empenho,Estimativo,ORIGINAL,25000,Ministério da Fazenda,...,Pregão,-8,LEI10520,ART01 LEI10520/02,-1,SEM INFORMACAO,8,8,1,170145000012019NE800007
462,64202138,193005112032019NE800007,2019NE800007,02/01/2019,NE,Nota de Empenho,Estimativo,CANCELAMENTO,53000,Ministério da Integração e do Desenvolvime,...,Pregão,-8,SI,SEM INFORMACAO,-1,SEM INFORMACAO,"166,37","166,37",1,193005112032019NE800007
504,71602050,130016000012019NE800007,2019NE800007,02/01/2019,NE,Nota de Empenho,Ordinário,CANCELAMENTO,22000,Ministério da Agricultura e Pecuária,...,Pregão,-8,SI,SEM INFORMACAO,-1,SEM INFORMACAO,10,10,1,130016000012019NE800007
518,66602028,153019152462019NE800007,2019NE800007,02/01/2019,NE,Nota de Empenho,Estimativo,CANCELAMENTO,26000,Ministério da Educação,...,Dispensa de Licitação,2,LEI 8666,ART24/02 LEI 8666/93,-1,SEM INFORMACAO,408,408,1,153019152462019NE800007
552,66302840,152005000012019NE800007,2019NE800007,02/01/2019,NE,Nota de Empenho,Global,ORIGINAL,26000,Ministério da Educação,...,Pregão,-8,LEI10520,ART01 LEI10520/02,-1,SEM INFORMACAO,7650,7650,1,152005000012019NE800007
661,63502095,170120000012019NE800007,2019NE800007,02/01/2019,NE,Nota de Empenho,Estimativo,CANCELAMENTO,25000,Ministério da Fazenda,...,Dispensa de Licitação,5,LEI 8666,ART24/05 LEI 8666/93,-1,SEM INFORMACAO,"10,58","10,58",1,170120000012019NE800007
698,68802193,170531000012019NE800007,2019NE800007,02/01/2019,NE,Nota de Empenho,Estimativo,ORIGINAL,46000,Ministério da Gestão e da Inovação em Ser,...,Pregão,-8,LEI10520,ART01 LEI10520/02,-1,SEM INFORMACAO,"250,15","250,15",1,170531000012019NE800007
932,63203166,158394264232019NE800007,2019NE800007,02/01/2019,NE,Nota de Empenho,Global,ORIGINAL,26000,Ministério da Educação,...,Pregão,-8,LEI10520,ART01 LEI10520/02,-1,SEM INFORMACAO,"2032,2","2032,2",1,158394264232019NE800007
965,69202067,155124264432019NE800007,2019NE800007,02/01/2019,NE,Nota de Empenho,Global,CANCELAMENTO,26000,Ministério da Educação,...,Pregão,-8,SI,SEM INFORMACAO,-1,SEM INFORMACAO,550,550,1,155124264432019NE800007


In [ ]:
result = df_2019[df_2019['IdEmpenho'].str.contains('800007', na=False)]
# elem_despesa_result = result['Elemento_to_str'].str[9:15]
result[['IdEmpenho', 'Cod_PT', 'IdOrgao', 'IdFonte', 'NrFonte', 'IdUnid', 'index']]

,CdFonteUG,IdEmpenho,Cod_PT,IdOrgao,IdFonte,NrFonte,IdUnid,index
583856,6,201900020068000070000000225,31909200,20190002006800008,2019006800006,10010000,680,810873


In [124]:
df.columns

Index(['index', 'Analise', 'IdEmpenho', 'Ano', 'Vlr_AnulacaoEmpenho',
       'CdFonteTCE', 'CdFonteUG', 'CNPJRaiz', 'CPFCNPJCredorQtNrs',
       'CPFCNPJCredor', 'Credor', 'DtEmpenho', 'DEFonteTCE', 'DEFonteUG',
       'DEPrograma', 'DEProjAtiv', 'DtAnomes', 'Elemento', 'ElemDespesaTCE',
       'ElemDespesaUG', 'Ente', 'Esfera', 'Funcao', 'Historico', 'IdContrato',
       'IdFonte', 'IdFuncao', 'Id_Orgao', 'IdPrograma', 'IdSubFuncao',
       'IdUnid', 'IdOrgao', 'NrFonte', 'NrFonteUG', 'NrLicitacao',
       'NrProjAtiv', 'NrEmpenho', 'ProgTrab', 'ProgTrabRed', 'ProjAtiv',
       'SubFuncao', 'Tp_Empenho', 'Unidade', 'Vlr_Empenho',
       'Vlr_Anul_Liquidacao', 'Vlr_Liquidacao', 'Vlr_Pagto', 'Vlr_Retencao',
       'Vlr_SubEmpenho', 'Vlr_Empenhado', 'Vlr_Liquidado', 'Vlr_Pago',
       'CGElem', 'CGProgTrab', 'CGigual', 'Cod_Elem', 'Cod_PT', 'CG',
       'CGtitulo', 'CGDesc', 'CGtitTCE', 'CGfreq', 'CGlevel', 'CGpai',
       'CGroot', 'CGchild', 'Elemento_to_str', 'Nr_Empenho_to_str'],
   